# Deploy Harman eNova Speech Recognition Model Package from AWS Marketplace 


Harman eNova Speech Recognition offers real-time transcription, built to scale for the enterprise. Optimized for CPU instances so you can focus on getting the insights you need from your voice data with minimal hardware requirement.

This sample notebook shows you how to deploy using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
    
- The input audio data should consist of complete audio files, rather than raw PCM data.
- The input audio sample should be 16000 or above. 
- The maximum audio file size for realtime inference is 20MB and for batch transform 50MB each file.

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page.
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [11]:
#This is sample ARN please replace with the subcribed ARN.
model_package_arn = "arn:aws:sagemaker:us-east-1:822940408628:model-package/marketplace-speech-recognition-1-0"

In [7]:
import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role
import time
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import time
import json 
import uuid
import boto3
import base64
import os

In [8]:
role = get_execution_role()

sagemaker_session = sage.Session()
runtime = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [9]:
model_name = "lifeware-marketplace-speech-recognition-model-ep"

content_type = "application/json"
accept = "text/plain"
real_time_inference_instance_type = 'ml.c5.xlarge'
batch_transform_inference_instance_type = 'ml.m4.16xlarge'

### A. Create an endpoint

In [10]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

Download Sample Input files https://github.com/HDTS-user/lifeware-speech-recognition

In [9]:
#Now we pull a sample input file for testing the model. For example we are refering to a .flac file which is local to 
#this notebook.
sample_audio_file = 'sample-audio-files/8297-275155-0032.flac'

<Add code snippet that shows the payload contents>

Now encode the audio in base64, this is the input format supported for inferencing with endpoints.

In [12]:
with open(sample_audio_file, mode='rb') as file:
        fileContent = file.read()
encodeddata = base64.b64encode(fileContent)

### C. Perform real-time inference

invoke the endpoint to get real time inference. For this example we are using "transcribe" task. Simillary same endpoint can be used for generating srt files.
Avialable Tasks: "transcribe", "transcribe_srt"

In [14]:
json_payload['wv_data'] =  encodeddata.decode('utf-8')
json_payload['task'] ="transcribe"
json_payload['lang'] ="en"
data = json.dumps(json_payload)

sm_runtime = boto3.client("sagemaker-runtime")
res = sm_runtime.invoke_endpoint(
    EndpointName=model_name,
    Body=data, 
    ContentType=content_type,  
    Accept=accept,  
)

### D. Visualize output

In [15]:
print(res['Body'].read().decode())

 While he was walking up and down the platform, with a mind doubly distressed by anxiety about his brother and anxiety about Sydney, the train from London came in.


Generate srt output

In [17]:
json_payload['wv_data'] =  encodeddata.decode('utf-8')
json_payload['task'] ="transcribe_srt"
json_payload['lang'] ="en"
data = json.dumps(json_payload)

sm_runtime = boto3.client("sagemaker-runtime")
res = sm_runtime.invoke_endpoint(
    EndpointName=model_name,
    Body=data,
    ContentType=content_type,  
    Accept=accept,  
)

print(res['Body'].read().decode())

1
00:00:00,000 --> 00:00:05,480
While he was walking up and down the platform with a mind doubly distressed by anxiety about
2
00:00:05,480 --> 00:00:35,440
his brother and anxiety about Sydney, the train from London came in.



### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [18]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [50]:
# upload the batch-transform job input files to S3
import boto3
import os

# Set the local path of the folder where sample audio files are kept
sample_audio_folder_path = 'sample-audio-files'

# Set the name of your S3 bucket and the name of the folder you want to create
bucket_name = 'sample-s3-bucket-for-speech-recog-test4'
input_folder_name = 'sample-audio'


# Create an S3 client
s3 = boto3.client('s3')

# Create the S3 bucket
s3.create_bucket(Bucket=bucket_name)

# Create the folder in the S3 bucket
s3.put_object(Bucket=bucket_name, Key=input_folder_name + '/')

# Upload a file to the folder in the S3 bucket
for dirName, subdirList, fileList in os.walk(sample_audio_folder_path):
    for fname in fileList:
        response = s3.upload_file(dirName+'/'+fname, bucket_name, input_folder_name + '/' + fname)
print("Transform input uploaded..")
#Display the content of S3 Bucket
# List the objects in the bucket
response = s3.list_objects_v2(
    Bucket=bucket_name
)

# Print the response
for obj in response['Contents']:
    print(obj['Key'])

Transform input uploaded..
sample-audio/
sample-audio/8297-275155-0031.flac
sample-audio/8297-275155-0032.flac


In [51]:
ModelName = "lifeware-marketplace-speech-recognition-model-batch-inference"
sm_boto3 = boto3.client("sagemaker")
cm_res_end = sm_boto3.create_model(
    ModelName=ModelName,  # name the of the model does not need to be the same as the image repob
     Containers=[
        {            
            'ModelPackageName': model_package_arn
        },
    ],    
    ExecutionRoleArn=get_execution_role(),
    EnableNetworkIsolation=False,
)
print("Model Arn: " + cm_res_end['ModelArn'])

Model Arn: arn:aws:sagemaker:us-east-1:822940408628:model/lifeware-marketplace-speech-recognition-model-batch-inference


Run Batch Transform Job

In [52]:
transformjobname= "lifeware-marketplace-speech-recognition"

In [53]:
# Run the batch-transform job
#optionl step to make uniue job name
transformjobname = transformjobname + time.strftime("-%Y%m%d%H%M", time.gmtime())

response_create_job = sm_boto3.create_transform_job(
    TransformJobName=transformjobname,
    ModelName=ModelName,
    MaxConcurrentTransforms=1,
    #InstanceCount=1
    MaxPayloadInMB=50,   
    Environment={
        'TASK': 'transcribe_srt',  # Creating srt output, it can be simple transcribe text also.
        'LANG': 'en'
    },
    TransformInput={
        'DataSource': {
            'S3DataSource': {                
                'S3DataType': 'S3Prefix', 
                'S3Uri': "s3://"+bucket_name+"/"    # Input Folder
            }
        },
        'ContentType':"audio/x-wav"
    },
    TransformOutput={
        'S3OutputPath': "s3://"+bucket_name+"/output/", #output folder
        'Accept': 'application/json',
        'AssembleWith': 'Line',        
    },
    TransformResources={
        'InstanceType': batch_transform_inference_instance_type,
        'InstanceCount': 1      
    }, 
)
print(response_create_job)
print("Output is available on following path:",s3audiooutput)

{'TransformJobArn': 'arn:aws:sagemaker:us-east-1:822940408628:transform-job/lifeware-marketplace-speech-recognition-202212151428', 'ResponseMetadata': {'RequestId': '9a97f4c1-6ff2-4bed-857a-bd4ac9297d07', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9a97f4c1-6ff2-4bed-857a-bd4ac9297d07', 'content-type': 'application/x-amz-json-1.1', 'content-length': '129', 'date': 'Thu, 15 Dec 2022 14:28:10 GMT'}, 'RetryAttempts': 0}}
Output is available on following path: s3://sample-s3-bucket-for-speech-recog-test2/output/


## 4. Clean-up

### A. Delete the model

In [37]:
sm_boto3.delete_model(ModelName=ModelName)

{'ResponseMetadata': {'RequestId': 'a6a99cd5-356f-4f8e-9b86-4f28912239fd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a6a99cd5-356f-4f8e-9b86-4f28912239fd',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 15 Dec 2022 13:34:31 GMT'},
  'RetryAttempts': 0}}

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

